missing values are in the columns:
- REPORT_TYPE
- STREET_DIRECTION
- STREET_NAME
- BEAT_OF_OCCURRENCE
- MOST_SEVERE_INJURY
- LATITUDE
- LONGITUDE
- LOCATION

i found replacement for:
- check integrate data crashes file for filled values

Column: REPORT_TYPE
Data Type: object
Unique Values: ['ON SCENE' 'NOT ON SCENE (DESK REPORT)' nan]

maybe report_type can be filled using the mode?